In [1]:
import ast
import os

import pandas as pd
from Bio import Entrez

from data_processing import analyze_exons
from fasta_processing import read_single_fasta, dict_align_to_fasta
from parse_psi_blast_results import parse_psiblast_xml, filter_psiblast_hits, update_df, save_files, \
    create_many_cassettes, dict_align_info_analyze

Entrez.email = "artemvaskaa@gmail.com"

## Переопределение функций

In [2]:
def find_cassettes(df):
    ref_exon_len = [37]
    found_protein_ids = {}

    for protein_id in df.protein_id:
        df_exons = analyze_exons(f"{dir}/{protein_id}/exons.fa")
        condition = set(ref_exon_len) & set(df_exons.length.tolist())
        if condition:
            found_protein_ids[protein_id] = condition
    df_found_protein_ids = df[df["protein_id"].isin(found_protein_ids.keys())]  # !!!

    data = {}
    for protein_id in df_found_protein_ids.protein_id:
        df_subset = df_found_protein_ids[df_found_protein_ids["protein_id"] == protein_id]
        org_name = df_subset.org_name.iloc[0]
        df_exons = analyze_exons(f"{dir}/{protein_id}/exons.fa")
        exon_37_idx = df_exons[df_exons["length"] == 37].index[0]
        exon_110_idx = exon_37_idx - 1
        data[f"{org_name}__{protein_id}"] = (df_exons, [exon_110_idx, exon_37_idx])

    return data, df_found_protein_ids

## Введение

In [3]:
# читаем все созданные таблицы
datasets_Cnidaria = pd.read_csv("../results_summary/datasets_Cnidaria_cds_cassette.tsv", sep="\t")
datasets_Ecdysozoa = pd.read_csv("../results_summary/datasets_Ecdysozoa_cds_cassette.tsv", sep="\t")
datasets_Spiralia = pd.read_csv("../results_summary/datasets_Spiralia_cds_cassette.tsv", sep="\t")

# объединяем их в одну
datasets_Protostomia = pd.concat([datasets_Cnidaria, datasets_Ecdysozoa, datasets_Spiralia], ignore_index=True)

# смотрим на value_counts в полученных данных из datasets, чтобы понять, что бластить
datasets_Protostomia.sub_phylum.value_counts()

sub_phylum
Endopterygota      31
Paraneoptera        9
Chelicerata         7
Crustacea           4
Bivalvia            3
Nematoda            3
Anthozoa            2
Gastropoda          2
Scalidophora        1
Polyneoptera        1
Platyhelminthes     1
Name: count, dtype: int64

## Platyhelminthes

[schistosoma_haematobium_platyhelminthes](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_051072204.1%20NXF1_3%20%5Borganism%3DSchistosoma%20haematobium%5D%20%5BGeneID%3D24590952%5D%0AMPHGNHRNKSNHYRWRSHGENDRRIDTSDEYLRRREHGGKFSSGRKSIISGNNVDMLKRAMNMNLIGGSA%0AASVVTAQNSGLAPGEVWVRITIVHGANHPMMDLQQLVTTIVGTQLRFYNTCVEGRNALMHAKIRQKDVQS%0AYRKSLQNLRDPSQGSQLITDITIVPEPRVPSSSDKRNESPNTSPLPETWIEALKQCFVQRYQPTTRSLDL%0ASSLHTDPVLLSQGLYLPLNKQAVVHTLITILKQNQAQLAVLNLSNNRLTHLNAFSPLSSTSAGFIPVSIE%0ARIDLSSNPLSSIPVLSGLRDIVGLVELDLTETPLMSKFNPNDKSFAAKLHTILPTIKRLNGQELPQTVQF%0AAIEQGSDSSKRPPTKPLPQSILGFFPNDEVKIALLSFLKLYLSRYDSKPRGESLLPYYTTVSQLVFSVSP%0AENRFPNSQNVSFTARVEIQNGSDQPTTAYLTTSRLNQAYFLRSRNLLRCRDQSRRRDMVVRGSLAIAHFL%0ADELPTTEHQLESLSVDVAFHSGTQMLFTMGGVFYEVSSMGSSSTNSSSHEKSVRKVLRCFTRTMILIAPG%0AGHIVQDDYIVSNPTTSLCKKYITEMATRCKQDSQASNQQQNVLSSDPSAPEVKENIVIEFSRRTGMNIPF%0ASRQCLEEYEWNANAALTAFETMNLAGKIPPEAFSV&JOB_TITLE=schistosoma_haematobium_platyhelminthes&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Platyhelminthes%20%28taxid%3A6157%29&EQ_MENU1=Schistosoma%20haematobium%20(taxid:6185)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&ADV_VIEW=on&BOOKMARK=on)

In [4]:
phylum = "Platyhelminthes"
dir = f"../Sequences_protein_id/{phylum}"

In [5]:
# # читаем результат BLAST
# name_of_blast_res = f"../Blast_res/psi_blast/schistosoma_haematobium_{phylum.lower()}.xml"
#
# # парсим, фильтруем и добавляем новые столбцы с координатами, именем организма и др.
# df = parse_psiblast_xml(name_of_blast_res)
# df = filter_psiblast_hits(df, min_qc=0, min_ident=0, min_sbjct_len=500)
# df = update_df(df)
#
# # создаем подпапку в заданной выше директории и сохраняем таблицу, чтобы потом не ждать
# os.makedirs(f"{dir}", exist_ok=True)
# df.to_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index=True)
#
# # закомментить после выполнения!!!

In [6]:
# сохраняем в заданную выше директорию
# save_files(df, dir)

# закомментить после выполнения!!!

In [7]:
# читаем сохраненную ранее таблицу
df = pd.read_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index_col=0,
                 converters={11: ast.literal_eval, 13: ast.literal_eval})

# получаем словарь для созданию кассет и protein_id для тех организмов, у кого нашелся экзон 37
data, df_found_protein_ids = find_cassettes(df)

# создаем кассеты и заодно получаем интроны для будущего анализа
introns = create_many_cassettes(dir, data)

In [8]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(df_found_protein_ids, phylum, "cds", dir)
df_cds.equal_to_cds.unique()

Mesocestoides_corti__VDD84209.1: start codon not in the beginning of sequence


array([ True])

In [9]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds
df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(df_found_protein_ids, phylum, "cds", dir)
df_cds_cassette

Mesocestoides_corti__VDD84209.1: start codon not in the beginning of sequence


,sub_phylum,org_name,org_name_protein_id,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
0,Platyhelminthes,Schistosoma_guineensis,Schistosoma_guineensis__CAH8656942.1,1995,True,-1,1996,652,239
1,Platyhelminthes,Schistosoma_curassoni,Schistosoma_curassoni__CAH8663826.1,1995,True,-1,1996,652,239
2,Platyhelminthes,Schistosoma_bovis,Schistosoma_bovis__CAH8671530.1,1995,True,-1,1996,652,239
3,Platyhelminthes,Schistosoma_margrebowiei,Schistosoma_margrebowiei__CAH8667553.1,1995,True,-1,1996,650,239
4,Platyhelminthes,Schistosoma_intercalatum,Schistosoma_intercalatum__CAH8646207.1,1995,True,-1,1996,652,239
5,Platyhelminthes,Schistosoma_rodhaini,Schistosoma_rodhaini__CAH8677847.1,1995,True,-1,1996,671,239
6,Platyhelminthes,Schistosoma_japonicum,Schistosoma_japonicum__TNN16962.1,1995,True,-1,1996,847,239
7,Platyhelminthes,Schistosoma_turkestanicum,Schistosoma_turkestanicum__CAH8587689.1,1995,True,-1,1996,905,239
8,Platyhelminthes,Paragonimus_westermani,Paragonimus_westermani__KAF8560478.1,2031,True,-1,2032,13971,239
9,Platyhelminthes,Taenia_crassiceps,Taenia_crassiceps__KAL5103217.1,1983,True,-1,1984,278,242


In [10]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,sub_phylum,org_name,org_name_protein_id,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
11,Platyhelminthes,Heterobilharzia_americana,Heterobilharzia_americana__CAH8668306.1,1995,True,-1,1996,2163,239
24,Platyhelminthes,Schistosoma_mattheei,Schistosoma_mattheei__CAH8638585.1,1935,True,-1,1936,649,239
26,Platyhelminthes,Schistosoma_mattheei,Schistosoma_mattheei__CAH8638513.1,1824,True,-1,1825,651,239
29,Platyhelminthes,Heterobilharzia_americana,Heterobilharzia_americana__CAH8650739.1,1626,True,-1,1627,2162,132


In [11]:
# удаляем дубликаты

# -----------------
# 29 - Heterobilharzia_americana__CAH8650739.1 - дубликат
# 26 - Schistosoma_mattheei__CAH8638513.1 - дубликат
# -----------------

indices_to_drop = [29, 26]
df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [12]:
# создаем папку и файлы для выравнивания
os.makedirs(f"../Alignment/psi_blast/{phylum}", exist_ok=True)

# закидываем в конец словаря референсную последовательность (вбить вручную)
dict_align_cds_cassette["Schistosoma_haematobium_reference"] = read_single_fasta("../Datasets/Spiralia/schistosoma_haematobium_1/ncbi_dataset/data/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.aln")

# закомментить последнюю строчку после выполнения!!!

In [13]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/psi_blast_{phylum}_cds_cassette.tsv", sep="\t", index=False)

## Mollusca/Bivalvia

[mya_arenaria](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_052767696.1%20LOC128208251%20%5Borganism%3DMya%20arenaria%5D%20%5BGeneID%3D128208251%5D%0AMAEFRVTTDRDGSRSFNDHDDRWTRGGGSGRYRGRGGNFNRRPRRGGGYGYGGSSYNRGRGGYSGGRGRG%0AGDRDGPGPRSRLDDDGDETMGNSETKDIARYNPYGRSSRGNFRGNSKYRSDNRNRPTQIGGDAGIFKRLG%0ALPLDRKTMGDSDWYKVTIPWGKKTEKDFILKSINDHIDVPFVPTYFHYEDKNAVFYVNDSRAAEGLKSTT%0AKRVTMPNGYKMTILVKPSQPPNIPMGKEEIDKLKVCMSNRYDPATKALNLSCLHTDQELAQSNLFMCLAR%0APQVMSNVVKVIKENIPELVQLDVSNNKLQSLEHLGGLVPSTPDMKVLNLANNKINMLEELRKVQKWKIDW%0ALTLDGNPLCDKFNDHTAYVSGVRRLFPKVLKLDTTDLPPPITFDIEARTDLPKSKDSYFPNDTVKNGVVK%0AFLKDYFLVYDSDDRTGLAGAYHETAMFSLSTSYNPTVQNKQTSLSTYIDETRNLLRVYKDTSRKFKTLKN%0AGNKIVAQLCLLPKTQHDPNSFVVDCNFATAQMISFNIQGVFKEVDKKSDKPPMRAFSRTFVTVASGSGMV%0AIVNDVLTVTNASPDQIQTAFKNPAPTPSSSPIPQTSPTEPFAAAGLTEIQTQMVASFMNDSRMNSEWSAK%0ACLVQNNWEYAKAGQNFLELQQKGLIPPEAFKT&JOB_TITLE=mya_arenaria_bivalvia&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Bivalvia%20%28taxid%3A6544%29&EQ_MENU1=Mya%20arenaria%20(taxid:6604)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NEW_VIEW=on&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&ADV_VIEW=on&BOOKMARK=on)

[crassostrea_virginica_bivalvia](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_052767696.1%20LOC128208251%20%5Borganism%3DMya%20arenaria%5D%20%5BGeneID%3D128208251%5D%0AMAEFRVTTDRDGSRSFNDHDDRWTRGGGSGRYRGRGGNFNRRPRRGGGYGYGGSSYNRGRGGYSGGRGRG%0AGDRDGPGPRSRLDDDGDETMGNSETKDIARYNPYGRSSRGNFRGNSKYRSDNRNRPTQIGGDAGIFKRLG%0ALPLDRKTMGDSDWYKVTIPWGKKTEKDFILKSINDHIDVPFVPTYFHYEDKNAVFYVNDSRAAEGLKSTT%0AKRVTMPNGYKMTILVKPSQPPNIPMGKEEIDKLKVCMSNRYDPATKALNLSCLHTDQELAQSNLFMCLAR%0APQVMSNVVKVIKENIPELVQLDVSNNKLQSLEHLGGLVPSTPDMKVLNLANNKINMLEELRKVQKWKIDW%0ALTLDGNPLCDKFNDHTAYVSGVRRLFPKVLKLDTTDLPPPITFDIEARTDLPKSKDSYFPNDTVKNGVVK%0AFLKDYFLVYDSDDRTGLAGAYHETAMFSLSTSYNPTVQNKQTSLSTYIDETRNLLRVYKDTSRKFKTLKN%0AGNKIVAQLCLLPKTQHDPNSFVVDCNFATAQMISFNIQGVFKEVDKKSDKPPMRAFSRTFVTVASGSGMV%0AIVNDVLTVTNASPDQIQTAFKNPAPTPSSSPIPQTSPTEPFAAAGLTEIQTQMVASFMNDSRMNSEWSAK%0ACLVQNNWEYAKAGQNFLELQQKGLIPPEAFKT&JOB_TITLE=mya_arenaria_bivalvia&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Bivalvia%20%28taxid%3A6544%29&EQ_MENU1=Mya%20arenaria%20(taxid:6604)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NEW_VIEW=on&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&ADV_VIEW=on&BOOKMARK=on)

[magallana_gigas_bivalvia](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_052767696.1%20LOC128208251%20%5Borganism%3DMya%20arenaria%5D%20%5BGeneID%3D128208251%5D%0AMAEFRVTTDRDGSRSFNDHDDRWTRGGGSGRYRGRGGNFNRRPRRGGGYGYGGSSYNRGRGGYSGGRGRG%0AGDRDGPGPRSRLDDDGDETMGNSETKDIARYNPYGRSSRGNFRGNSKYRSDNRNRPTQIGGDAGIFKRLG%0ALPLDRKTMGDSDWYKVTIPWGKKTEKDFILKSINDHIDVPFVPTYFHYEDKNAVFYVNDSRAAEGLKSTT%0AKRVTMPNGYKMTILVKPSQPPNIPMGKEEIDKLKVCMSNRYDPATKALNLSCLHTDQELAQSNLFMCLAR%0APQVMSNVVKVIKENIPELVQLDVSNNKLQSLEHLGGLVPSTPDMKVLNLANNKINMLEELRKVQKWKIDW%0ALTLDGNPLCDKFNDHTAYVSGVRRLFPKVLKLDTTDLPPPITFDIEARTDLPKSKDSYFPNDTVKNGVVK%0AFLKDYFLVYDSDDRTGLAGAYHETAMFSLSTSYNPTVQNKQTSLSTYIDETRNLLRVYKDTSRKFKTLKN%0AGNKIVAQLCLLPKTQHDPNSFVVDCNFATAQMISFNIQGVFKEVDKKSDKPPMRAFSRTFVTVASGSGMV%0AIVNDVLTVTNASPDQIQTAFKNPAPTPSSSPIPQTSPTEPFAAAGLTEIQTQMVASFMNDSRMNSEWSAK%0ACLVQNNWEYAKAGQNFLELQQKGLIPPEAFKT&JOB_TITLE=mya_arenaria_bivalvia&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Bivalvia%20%28taxid%3A6544%29&EQ_MENU1=Mya%20arenaria%20(taxid:6604)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NEW_VIEW=on&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&ADV_VIEW=on&BOOKMARK=on)

#### mya_arenaria

In [14]:
phylum = "Bivalvia"
dir = f"../Sequences_protein_id/{phylum}"

In [15]:
# # читаем результат BLAST
# name_of_blast_res = f"../Blast_res/psi_blast/schistosoma_haematobium_{phylum.lower()}.xml"
#
# # парсим, фильтруем и добавляем новые столбцы с координатами, именем организма и др.
# df = parse_psiblast_xml(name_of_blast_res)
# df = filter_psiblast_hits(df, min_qc=0, min_ident=0, min_sbjct_len=500)
# df = update_df(df)
#
# # создаем подпапку в заданной выше директории и сохраняем таблицу, чтобы потом не ждать
# os.makedirs(f"{dir}", exist_ok=True)
# df.to_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index=True)
#
# # закомментить после выполнения!!!

In [16]:
# сохраняем в заданную выше директорию
# save_files(df, dir)

# закомментить после выполнения!!!

In [17]:
# читаем сохраненную ранее таблицу
df = pd.read_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index_col=0,
                 converters={11: ast.literal_eval, 13: ast.literal_eval})

# получаем словарь для созданию кассет и protein_id для тех организмов, у кого нашелся экзон 37
data, df_found_protein_ids = find_cassettes(df)

# создаем кассеты и заодно получаем интроны для будущего анализа
introns = create_many_cassettes(dir, data)

In [18]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(df_found_protein_ids, phylum, "cds", dir)
df_cds.equal_to_cds.unique()

Dreissena_polymorpha__KAH3852511.1: start codon not in the beginning of sequence
Sinanodonta_woodiana__KAL3874362.1: no stop codon found


array([ True])

In [19]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds
df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(df_found_protein_ids, phylum, "cds", dir)
df_cds_cassette

Dreissena_polymorpha__KAH3852511.1: start codon not in the beginning of sequence
Sinanodonta_woodiana__KAL3874362.1: no stop codon found


,sub_phylum,org_name,org_name_protein_id,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
0,Bivalvia,Mercenaria_mercenaria,Mercenaria_mercenaria__XP_053373641.1,1983,True,-1,1984,1690,110
1,Bivalvia,Dreissena_polymorpha,Dreissena_polymorpha__XP_052275796.1,1989,True,-1,1990,2207,110
2,Bivalvia,Ruditapes_philippinarum,Ruditapes_philippinarum__XP_060562241.1,1971,True,-1,1972,1646,110
3,Bivalvia,Mactra_antiquata,Mactra_antiquata__KAL4236273.1,1722,True,-1,1723,2319,110
4,Bivalvia,Mytilus_coruscus,Mytilus_coruscus__CAC5418521.1,1977,True,-1,1978,1234,110
5,Bivalvia,Potamilus_streckersoni,Potamilus_streckersoni__KAK3583320.1,1965,True,-1,1966,4567,110
6,Bivalvia,Mytilus_californianus,Mytilus_californianus__XP_052058679.1,1977,True,-1,1978,1248,110
7,Bivalvia,Saccostrea_echinata,Saccostrea_echinata__XP_061162409.1,1986,True,-1,1987,1556,110
8,Bivalvia,Mytilus_edulis,Mytilus_edulis__CAG2249952.1,1977,True,-1,1978,1360,110
9,Bivalvia,Mytilus_edulis,Mytilus_edulis__XP_071161937.1,1977,True,-1,1978,1360,110


In [20]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,sub_phylum,org_name,org_name_protein_id,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
5,Bivalvia,Potamilus_streckersoni,Potamilus_streckersoni__KAK3583320.1,1965,True,-1,1966,4567,110
6,Bivalvia,Mytilus_californianus,Mytilus_californianus__XP_052058679.1,1977,True,-1,1978,1248,110
8,Bivalvia,Mytilus_edulis,Mytilus_edulis__CAG2249952.1,1977,True,-1,1978,1360,110
9,Bivalvia,Mytilus_edulis,Mytilus_edulis__XP_071161937.1,1977,True,-1,1978,1360,110
19,Bivalvia,Argopecten_irradians,Argopecten_irradians__XP_069116938.1,2058,True,-1,2059,5057,110
20,Bivalvia,Argopecten_irradians,Argopecten_irradians__XP_069116939.1,2058,True,-1,2059,5057,110
21,Bivalvia,Magallana_gigas,Magallana_gigas__XP_034325678.2,1929,True,-1,1930,1537,110
22,Bivalvia,Magallana_gigas,Magallana_gigas__XP_065942772.1,1929,True,-1,1930,1537,110
23,Bivalvia,Sinanodonta_woodiana,Sinanodonta_woodiana__KAL3887057.1,1977,True,-1,1978,6312,101
24,Bivalvia,Potamilus_streckersoni,Potamilus_streckersoni__KAK3601259.1,1971,True,-1,1972,5690,101


In [21]:
# удаляем дубликаты

# -----------------
# 19 - Argopecten_irradians__XP_069116938.1 - дубликат
# 21 - Magallana_gigas__XP_034325678.2 - дубликат
# 6 - Mytilus_californianus__XP_052058679.1 - дубликат
# 8 - Mytilus_edulis__CAG2249952.1 - дубликат
# 24 - Potamilus_streckersoni__KAK3601259.1 - дубликат
# 23, 25 - Sinanodonta_woodiana__KAL3887057.1, Sinanodonta_woodiana__KAL3887113.1 - трипликат
# -----------------

indices_to_drop = [19, 21, 6, 8, 24, 23, 25]
df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [22]:
# создаем папку и файлы для выравнивания
os.makedirs(f"../Alignment/psi_blast/{phylum}", exist_ok=True)

# закидываем в конец словаря референсную последовательность (вбить вручную)
dict_align_cds_cassette["Mya_arenaria_reference"] = read_single_fasta("../Datasets/Bivalvia/mya_arenaria_0/ncbi_dataset/data/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.aln")

# закомментить последнюю строчку после выполнения!!!

In [23]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/psi_blast_{phylum}_cds_cassette.tsv", sep="\t", index=False)

## Mollusca/Gastropoda

[gigantopelta_aegis](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_041353365.1%20LOC121371499%20%5Borganism%3DGigantopelta%20aegis%5D%20%5BGeneID%3D121371499%5D%0AMSSLTVTAGRDGSRSFGGHDDRWSGKQRNQRRGRGKSYYSRGSYNRGGNKGPRGGGVNPRSRFTDEDDDV%0ANMDSSSGHINRRFDPYGQGGGWNRRNRGRGSFKGGRYHGDFQSHKDTGAMQRMGLPVSRQNQWYKITIPH%0AGKKTGKDEILKLINSVIGAPFQPVYFHYQNKDALFYVKDWDQATALRRTSKMITLPSGFKMIVVVSPCSP%0APVIPMDKESIEKLQKRMSERYDPASKCLDLSSLYQDEVLSASGLYLALNRANTMSNVVKIIQENIPELVG%0ALDLSSNRLLSLSHMTDLVSAAPFVTKLNIGKNQLRSIEELQKIEGWKLLQLVLDGNDLCDRYKERSEYIS%0AMVRKRFPKVINLDGHELPPPITFDLETTTVIPPSKGSYFMNDEVKTIVVKFLKEYYTIYDSDNRQPLFEA%0AYHEQAVFCIATAYNGTLDYKQPSLTDYLQESRNIFRVKDTARREKSIKNGRLPVVSQLCLLPKTTHDPNS%0AFVVDINIVRPTLLSFSLTGIFKETESKSDKPPIRAFNRLFVTVPCGSGMVITNDVLTITNASPEQSQSAF%0AKSTGPTPSSSPVSSAPPAVPSTSLSPIPGTSAVPNKEMIQRFSAESGMNAEWSHKCLEENGWNYKKAALV%0AFTELHSQGKIPQDAFVK&JOB_TITLE=gigantopelta_aegis_gastropoda&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Gastropoda%20%28taxid%3A6448%29&EQ_MENU1=Gigantopelta%20aegis%20(taxid:1735272)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NCBI_GI=false&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&SHOW_CDS_FEATURE=false&ADV_VIEW=on&BOOKMARK=on)

In [24]:
phylum = "Gastropoda"
dir = f"../Sequences_protein_id/{phylum}"

In [25]:
# # читаем результат BLAST
# name_of_blast_res = f"../Blast_res/psi_blast/schistosoma_haematobium_{phylum.lower()}.xml"
#
# # парсим, фильтруем и добавляем новые столбцы с координатами, именем организма и др.
# df = parse_psiblast_xml(name_of_blast_res)
# df = filter_psiblast_hits(df, min_qc=0, min_ident=0, min_sbjct_len=500)
# df = update_df(df)
#
# # создаем подпапку в заданной выше директории и сохраняем таблицу, чтобы потом не ждать
# os.makedirs(f"{dir}", exist_ok=True)
# df.to_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index=True)
#
# # закомментить после выполнения!!!

In [26]:
# сохраняем в заданную выше директорию
# save_files(df, dir)

# закомментить после выполнения!!!

In [27]:
# читаем сохраненную ранее таблицу
df = pd.read_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index_col=0,
                 converters={11: ast.literal_eval, 13: ast.literal_eval})

# получаем словарь для созданию кассет и protein_id для тех организмов, у кого нашелся экзон 37
data, df_found_protein_ids = find_cassettes(df)

# создаем кассеты и заодно получаем интроны для будущего анализа
introns = create_many_cassettes(dir, data)

In [28]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(df_found_protein_ids, phylum, "cds", dir)
df_cds.equal_to_cds.unique()

Physella_acuta__XP_059147995.1: start codon not in the beginning of sequence
Candidula_unifasciata__CAG5136897.1: start codon not in the beginning of sequence


array([ True])

In [29]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds
df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(df_found_protein_ids, phylum, "cds", dir)
df_cds_cassette

Physella_acuta__XP_059147995.1: start codon not in the beginning of sequence
Candidula_unifasciata__CAG5136897.1: start codon not in the beginning of sequence


,sub_phylum,org_name,org_name_protein_id,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
0,Gastropoda,Haliotis_cracherodii,Haliotis_cracherodii__XP_071079091.1,1914,True,-1,1915,2506,110
1,Gastropoda,Haliotis_rufescens,Haliotis_rufescens__XP_046327842.1,1914,True,-1,1915,2505,110
2,Gastropoda,Haliotis_asinina,Haliotis_asinina__XP_067680995.1,1911,True,-1,1912,2375,110
3,Gastropoda,Patella_vulgata,Patella_vulgata__XP_055957145.1,2031,True,-1,2032,1384,110
4,Gastropoda,Patella_caerulea,Patella_caerulea__KAK6181635.1,2004,True,-1,2005,1362,110
5,Gastropoda,Patella_caerulea,Patella_caerulea__KAK6181636.1,2004,True,-1,2005,1362,110
6,Gastropoda,Littorina_saxatilis,Littorina_saxatilis__XP_070180466.1,2028,True,-1,2029,6746,218
7,Gastropoda,Batillaria_attramentaria,Batillaria_attramentaria__KAK7466614.1,1935,True,-1,1936,8614,110
8,Gastropoda,Lymnaea_stagnalis,Lymnaea_stagnalis__CAL1544423.1,1995,True,-1,1996,2705,221
9,Gastropoda,Pomacea_canaliculata,Pomacea_canaliculata__PVD26032.1,3012,True,-1,3013,255,56


In [30]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,sub_phylum,org_name,org_name_protein_id,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
4,Gastropoda,Patella_caerulea,Patella_caerulea__KAK6181635.1,2004,True,-1,2005,1362,110
5,Gastropoda,Patella_caerulea,Patella_caerulea__KAK6181636.1,2004,True,-1,2005,1362,110
14,Gastropoda,Bulinus_truncatus,Bulinus_truncatus__KAH9514235.1,2025,True,-1,2026,1873,221
15,Gastropoda,Biomphalaria_glabrata,Biomphalaria_glabrata__XP_055880571.1,1863,True,-1,1864,1889,221
16,Gastropoda,Bulinus_truncatus,Bulinus_truncatus__KAH9514233.1,2544,True,-1,2545,1873,221
18,Gastropoda,Biomphalaria_glabrata,Biomphalaria_glabrata__XP_055880572.1,1863,True,-1,1864,1889,221


In [31]:
# удаляем дубликаты

# -----------------
# 15 - Biomphalaria_glabrata__XP_055880571.1 - дубликат
# 16 - Bulinus_truncatus__KAH9514233.1 - дубликат
# 4 - Patella_caerulea__KAK6181635.1 - дубликат
# -----------------

indices_to_drop = [15, 16, 4]
df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [32]:
# создаем папку и файлы для выравнивания
os.makedirs(f"../Alignment/psi_blast/{phylum}", exist_ok=True)

# закидываем в конец словаря референсную последовательность (вбить вручную)
dict_align_cds_cassette["Gigantopelta_aegis_reference"] = read_single_fasta("../Datasets/Gastropoda/gigantopelta_aegis_1/ncbi_dataset/data/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.aln")

# закомментить последнюю строчку после выполнения!!!

In [33]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/psi_blast_{phylum}_cds_cassette.tsv", sep="\t", index=False)

## Ecdysozoa/Nematoda

[caenorhabditis_elegans](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3ENP_001129880.1%20nxf-1%20%5Borganism%3DCaenorhabditis%20elegans%5D%20%5BGeneID%3D191736%5D%0AMNRKGFGGHRDAKQLSRTKNRFARLDPDTQSRYEDDDEPAVPVRASLTSASSRGRGGSSRGFGQSAASIA%0ANTGVRNADIVYKCRATGAAKKVDAKWLIKQLNQIIENFKPLLWTDNARGDFEWYVRDEDTASTIRANNRR%0AVVHKESGTRVEFYTSKVPAPWMKLKREEIEIIHRVVDKRHNAENRVLDLSNFHEDEEFKAKDMMMNLTKG%0ANVMLTVLDHIDDKYGNIVALSLSNNRIRHLDYASALVSIAKFVMELDLSHNHISTEKELEKFAGLPVERF%0AFFEGNPVVESFTQRAAYISYIHQSFPRCNMLDGVEVQPLVVGPDLDIHDAMPFRAGYYPNPQIRVLVEQF%0AVTSYFDFYDGPDGQRTRRNLHNAYDADASTFSLTIEHLRGSSHARHHNDECFAQYAGVSHNVLKQERFAR%0AHRASRSARGAMDIAVALSKLPTSSHMRDTFIVDVFLQSNDLLGFTVQGLFCDGDLTQTPSPSFFSRSFLV%0ASPRENDSVAVISDQLFITVASLDRLEKFKKLYDQSIANGAAVEQVSAVQIAQIGVNGMGFDGAPALPIRE%0AEMIKAMCQFSGMIPPFSEKCLADCAWNFDFACQKFNEIKSSVPAEAFAH&JOB_TITLE=caenorhabditis_elegans_nematoda&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Nematoda%20%28taxid%3A6231%29&EQ_MENU1=Caenorhabditis%20elegans%20(taxid:6239)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NCBI_GI=false&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&SHOW_CDS_FEATURE=false&ADV_VIEW=on&BOOKMARK=on)

In [34]:
phylum = "Nematoda"
dir = f"../Sequences_protein_id/{phylum}"

In [35]:
# # читаем результат BLAST
# name_of_blast_res = f"../Blast_res/psi_blast/schistosoma_haematobium_{phylum.lower()}.xml"
#
# # парсим, фильтруем и добавляем новые столбцы с координатами, именем организма и др.
# df = parse_psiblast_xml(name_of_blast_res)
# df = filter_psiblast_hits(df, min_qc=0, min_ident=0, min_sbjct_len=500)
# df = update_df(df)
#
# # создаем подпапку в заданной выше директории и сохраняем таблицу, чтобы потом не ждать
# os.makedirs(f"{dir}", exist_ok=True)
# df.to_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index=True)
#
# # закомментить после выполнения!!!

In [36]:
# сохраняем в заданную выше директорию
# save_files(df, dir)

# закомментить после выполнения!!!

In [37]:
# читаем сохраненную ранее таблицу
df = pd.read_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index_col=0,
                 converters={11: ast.literal_eval, 13: ast.literal_eval})

# получаем словарь для созданию кассет и protein_id для тех организмов, у кого нашелся экзон 37
data, df_found_protein_ids = find_cassettes(df)

# создаем кассеты и заодно получаем интроны для будущего анализа
introns = create_many_cassettes(dir, data)

In [38]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(df_found_protein_ids, phylum, "cds", dir)
df_cds.equal_to_cds.unique()

Angiostrongylus_cantonensis__KAE9413040.1: no stop codon found
Mesorhabditis_belari__CAJ0960930.1: no stop codon found
Pristionchus_fissidentatus__GMT09523.1: start codon not in the beginning of sequence
Mesorhabditis_spiculigera__CAJ0568410.1: no stop codon found
Pristionchus_pacificus__KAF8386643.1: no stop codon found
Pristionchus_entomophagus__GMS77887.1: start codon not in the beginning of sequence
Pristionchus_pacificus__KAF8382567.1: start codon not in the beginning of sequence
Onchocerca_ochengi__VDK67817.1: no stop codon found
Aphelenchoides_avenae__KAH7707721.1: start codon not in the beginning of sequence


array([ True])

In [39]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds
df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(df_found_protein_ids, phylum, "cds", dir)
df_cds_cassette

Angiostrongylus_cantonensis__KAE9413040.1: no stop codon found
Mesorhabditis_belari__CAJ0960930.1: no stop codon found
Pristionchus_fissidentatus__GMT09523.1: start codon not in the beginning of sequence
Mesorhabditis_spiculigera__CAJ0568410.1: no stop codon found
Pristionchus_pacificus__KAF8386643.1: no stop codon found
Pristionchus_entomophagus__GMS77887.1: start codon not in the beginning of sequence
Pristionchus_pacificus__KAF8382567.1: start codon not in the beginning of sequence
Onchocerca_ochengi__VDK67817.1: no stop codon found
Aphelenchoides_avenae__KAH7707721.1: start codon not in the beginning of sequence


,sub_phylum,org_name,org_name_protein_id,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
0,Nematoda,Caenorhabditis_nigoni,Caenorhabditis_nigoni__PIC28742.1,1857,True,-1,1858,142,110
1,Nematoda,Caenorhabditis_briggsae,Caenorhabditis_briggsae__UMM37086.1,1857,True,-1,1858,145,110
2,Nematoda,Caenorhabditis_brenneri,Caenorhabditis_brenneri__EGT34035.1,1857,True,-1,1858,130,110
3,Nematoda,Caenorhabditis_brenneri,Caenorhabditis_brenneri__EGT56062.1,1905,True,-1,1906,130,245
4,Nematoda,Caenorhabditis_sp._36_PRJEB53466,Caenorhabditis_sp._36_PRJEB53466__CAI2353387.1,1860,True,-1,1861,133,110
5,Nematoda,Caenorhabditis_angaria,Caenorhabditis_angaria__CAI5450371.1,1854,True,-1,1855,96,110
6,Nematoda,Caenorhabditis_bovis,Caenorhabditis_bovis__CAB3397759.1,1860,True,-1,1861,235,110
7,Nematoda,Caenorhabditis_bovis,Caenorhabditis_bovis__CAB3397758.1,1860,True,-1,1861,235,110
8,Nematoda,Caenorhabditis_auriculariae,Caenorhabditis_auriculariae__CAD6196466.1,1845,True,-1,1846,156,110
9,Nematoda,Necator_americanus,Necator_americanus__ETN84757.1,1965,True,-1,1966,243,110


In [40]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,sub_phylum,org_name,org_name_protein_id,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
2,Nematoda,Caenorhabditis_brenneri,Caenorhabditis_brenneri__EGT34035.1,1857,True,-1,1858,130,110
3,Nematoda,Caenorhabditis_brenneri,Caenorhabditis_brenneri__EGT56062.1,1905,True,-1,1906,130,245
6,Nematoda,Caenorhabditis_bovis,Caenorhabditis_bovis__CAB3397759.1,1860,True,-1,1861,235,110
7,Nematoda,Caenorhabditis_bovis,Caenorhabditis_bovis__CAB3397758.1,1860,True,-1,1861,235,110
10,Nematoda,Nippostrongylus_brasiliensis,Nippostrongylus_brasiliensis__WKY10434.1,1824,True,-1,1825,235,110
12,Nematoda,Nippostrongylus_brasiliensis,Nippostrongylus_brasiliensis__VDL77853.1,1911,True,-1,1912,235,110
13,Nematoda,Haemonchus_contortus,Haemonchus_contortus__XGW01888.1,2034,True,-1,2035,220,110
14,Nematoda,Haemonchus_contortus,Haemonchus_contortus__XGW01889.1,2034,True,-1,2035,220,110
19,Nematoda,Diploscapter_pachys,Diploscapter_pachys__PAV87020.1,1914,True,-1,1915,662,110
21,Nematoda,Nippostrongylus_brasiliensis,Nippostrongylus_brasiliensis__WKY10433.1,1824,True,-1,1825,235,110


In [41]:
# удаляем дубликаты

# -----------------
# 30 - Acanthocheilonema_viteae__KAL4002522.1 - дубликат
# 46, 45 - Aphelenchoides_avenae__KAH7716608.1, Aphelenchoides_avenae__KAH7703299.1 - трипликат
# 38 - Brugia_pahangi__VDN82903.1 - дубликат
# 7 - Caenorhabditis_bovis__CAB3397758.1 - дубликат
# 3 - Caenorhabditis_brenneri__EGT56062.1 - дубликат
# 23 - Diploscapter_pachys__PAV81848.1 - дубликат
# 34 - Ditylenchus_destructor__KAI1727416.1 - дубликат
# 13 - Haemonchus_contortus__XGW01888.1 - дубликат
# 12, 22 - Nippostrongylus_brasiliensis__VDL77853.1, Nippostrongylus_brasiliensis__WKY10433.1 - трипликат
# 49 - Steinernema_carpocapsae__TKR60559.1 - дубликат
# 39 - Wuchereria_bancrofti__VDM22199.1 - дубликат
# -----------------

indices_to_drop = [30, 46, 45, 38, 7, 3, 23, 34, 13, 12, 22, 49, 39]
df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [42]:
# создаем папку и файлы для выравнивания
os.makedirs(f"../Alignment/psi_blast/{phylum}", exist_ok=True)

# закидываем в конец словаря референсную последовательность (вбить вручную)
dict_align_cds_cassette["Caenorhabditis_elegans_reference"] = read_single_fasta("../Datasets/Nematoda/caenorhabditis_elegans_0/ncbi_dataset/data/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.aln")

# закомментить последнюю строчку после выполнения!!!

In [43]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/psi_blast_{phylum}_cds_cassette.tsv", sep="\t", index=False)

## Ecdysozoa/Scalidophora

[priapulus_caudatus](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_014677623.1%20LOC106817467%20%5Borganism%3DPriapulus%20caudatus%5D%20%5BGeneID%3D106817467%5D%0AMSYQITTNRDGDRNFYGNRGRRRGRGGNGNQSGGGPTPRSRLDAVDEEGDIDMGGGAGRQVHFNRYTPYG%0AGGGGRRPYRGGPRGDRGGYRGNSSYQPPSTMSDRAGSSRNSLERLGLPLQRGGRRGSGNYNKDRHDERQR%0ASVWYKIMIPYGREMGKDFILRSLTTECSVPFTPISFRLDGKMAIFNVEGWDTAEALMKVSRLVTAPSGFK%0AMLVKATPSGPPMPQNLSPTILEQVKEVLNKRYNSELKTLNLTDFHNDPDFKAKKLYLPLNRQNVILSVID%0ALIGKTASEVVALDFTNNKLPNCDSISTLYTKTPKLKGLNLSKNYIRSEYDLDRIKDLNLQELILEGNPLC%0ASQFKDKMSYVSAIRKRFPKVIKLDGLDLPPPIGFDVETHLELPKSKESFFVNDVVKTLVLRFIQEYFMVY%0ADSNDRQQLLNAYHEQAFFSMFAVRKDNSRGPYLQEYLSESRNMIKVKGFERRSKLLKRGRLAVVSMLTTL%0APKTQHDPTSFCIDINHYSAQLLSFTVTGVFKEGDKSSPPVRSFTRVFVVVPVGDGLCIVNEQMCINNATV%0AEQVQASFKSPTPTPSSSPSYPQPGPSGLTSSPVTGAPLVAVNSPSSLTVEQKKVMVEQFMKDSGMNSEWS%0AAKCLEENAWNYEGAGQVFLELRKVGSIPMEAFTKS&JOB_TITLE=priapulus_caudatus_scalidophora&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Scalidophora%20%28taxid%3A1215728%29&EQ_MENU1=Priapulus%20caudatus%20(taxid:37621)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NCBI_GI=false&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&SHOW_CDS_FEATURE=false&ADV_VIEW=on&BOOKMARK=on)

No significant similarity found.

## Cnidaria/Anthozoa

[nematostella_vectensis](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_001631177.1%20LOC5510737%20%5Borganism%3DNematostella%20vectensis%5D%20%5BGeneID%3D5510737%5D%0AMAGIFGRAMKDVAFSVTTSGDGSRTFNSNIGDGMEDDRQSNRGRGRGGYNNNQGSYRSRGRGRGRGRGRG%0AGNNRGRGGRGGRQDANPRSYLADEEDDESMGDADDNTGGYTSRYTPYGARPPSRRGYQHDDRSGGAGGGI%0AKSRLGQRQHSGWKSFDSDWHRVVVNRAKIHDKEWLIKRLQSSSEEAFQPVEFHYMGESAVFFVEGSRAAE%0AALKRVSHSITVKDGSKLIITVRPSQKPFKSHKGGAGGGGGSSQWSPENEQVLKECLSNRYNPQTKTMDLT%0ADMFHDEVLKANNVFGALNKYPLAQEILKLIGENCPDVESLDVSNNRLFQLDHFKDLATQAPGIKCLNLSN%0ATMLKSIDELDKVKGLKELVTLNLTGNTFCKSFEGKSSAYVSAVRSRFPKVTNLDGNELPPPIGFDLQTST%0AVLPTVQGSYIPDPAVKDLVLKFLEQYFKIYDSGDRQPLLDAYHDQAIFSMCVDTQANSSTKGSRGPSLGP%0AYMRNSRNMKRVTETDHRAALIKHNRLSVVAMLNDFPGTKHDLASFVVDINLTLSTIISFSVRGLFMEEDK%0ATARSFTRVFVAVPAAGGKALSIINDELHIRNASSSQAEKLASAASASIITPNVGSGIMAPAPSATPLPAA%0ATTVPAAGMQNLTPQQQQQMVLQFSNQSQMNPEWSFKCLSENGWNYEKSAEIFTSLQAAGSIPPDAFVKK&JOB_TITLE=nematostella_vectensis_anthozoa&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Anthozoa%20%28taxid%3A6101%29&EQ_MENU1=Nematostella%20vectensis%20(taxid:45351)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NCBI_GI=false&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&SHOW_CDS_FEATURE=false&ADV_VIEW=on&BOOKMARK=on)

In [44]:
phylum = "Anthozoa"
dir = f"../Sequences_protein_id/{phylum}"

In [45]:
# # читаем результат BLAST
# name_of_blast_res = f"../Blast_res/psi_blast/schistosoma_haematobium_{phylum.lower()}.xml"
#
# # парсим, фильтруем и добавляем новые столбцы с координатами, именем организма и др.
# df = parse_psiblast_xml(name_of_blast_res)
# df = filter_psiblast_hits(df, min_qc=0, min_ident=0, min_sbjct_len=500)
# df = update_df(df)
#
# # создаем подпапку в заданной выше директории и сохраняем таблицу, чтобы потом не ждать
# os.makedirs(f"{dir}", exist_ok=True)
# df.to_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index=True)
#
# # закомментить после выполнения!!!

In [46]:
# сохраняем в заданную выше директорию
# save_files(df, dir)

# закомментить после выполнения!!!

In [47]:
# читаем сохраненную ранее таблицу
df = pd.read_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index_col=0,
                 converters={11: ast.literal_eval, 13: ast.literal_eval})

# получаем словарь для созданию кассет и protein_id для тех организмов, у кого нашелся экзон 37
data, df_found_protein_ids = find_cassettes(df)

# создаем кассеты и заодно получаем интроны для будущего анализа
introns = create_many_cassettes(dir, data)

In [48]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(df_found_protein_ids, phylum, "cds", dir)
df_cds.equal_to_cds.unique()

Exaiptasia_diaphana__XP_020894941.1: start codon not in the beginning of sequence
Porites_lobata__CAH3150435.1: start codon not in the beginning of sequence


array([ True])

In [49]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds
df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(df_found_protein_ids, phylum, "cds", dir)
df_cds_cassette

Exaiptasia_diaphana__XP_020894941.1: start codon not in the beginning of sequence
Porites_lobata__CAH3150435.1: start codon not in the beginning of sequence


,sub_phylum,org_name,org_name_protein_id,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
0,Anthozoa,Actinia_tenebrosa,Actinia_tenebrosa__XP_031555922.1,2091,True,-1,2092,173,116
1,Anthozoa,Pocillopora_verrucosa,Pocillopora_verrucosa__XP_058954564.2,1995,True,-1,1996,390,116
2,Anthozoa,Porites_lutea,Porites_lutea__XP_073247758.1,1986,True,-1,1987,711,116
3,Anthozoa,Pocillopora_damicornis,Pocillopora_damicornis__XP_027046453.1,1998,True,-1,1999,392,116
4,Anthozoa,Porites_lutea,Porites_lutea__XP_073247755.1,1986,True,-1,1987,711,116
5,Anthozoa,Porites_lutea,Porites_lutea__XP_073247757.1,1986,True,-1,1987,711,116
6,Anthozoa,Pocillopora_damicornis,Pocillopora_damicornis__XP_027046452.1,1998,True,-1,1999,392,116
7,Anthozoa,Montipora_foliosa,Montipora_foliosa__XP_068703612.1,1992,True,-1,1993,907,116
8,Anthozoa,Montipora_foliosa,Montipora_foliosa__XP_068703613.1,1992,True,-1,1993,907,116
9,Anthozoa,Pocillopora_verrucosa,Pocillopora_verrucosa__XP_058954563.2,1995,True,-1,1996,390,116


In [50]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,sub_phylum,org_name,org_name_protein_id,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
1,Anthozoa,Pocillopora_verrucosa,Pocillopora_verrucosa__XP_058954564.2,1995,True,-1,1996,390,116
2,Anthozoa,Porites_lutea,Porites_lutea__XP_073247758.1,1986,True,-1,1987,711,116
3,Anthozoa,Pocillopora_damicornis,Pocillopora_damicornis__XP_027046453.1,1998,True,-1,1999,392,116
4,Anthozoa,Porites_lutea,Porites_lutea__XP_073247755.1,1986,True,-1,1987,711,116
5,Anthozoa,Porites_lutea,Porites_lutea__XP_073247757.1,1986,True,-1,1987,711,116
6,Anthozoa,Pocillopora_damicornis,Pocillopora_damicornis__XP_027046452.1,1998,True,-1,1999,392,116
7,Anthozoa,Montipora_foliosa,Montipora_foliosa__XP_068703612.1,1992,True,-1,1993,907,116
8,Anthozoa,Montipora_foliosa,Montipora_foliosa__XP_068703613.1,1992,True,-1,1993,907,116
9,Anthozoa,Pocillopora_verrucosa,Pocillopora_verrucosa__XP_058954563.2,1995,True,-1,1996,390,116
11,Anthozoa,Acropora_millepora,Acropora_millepora__XP_029179300.2,1995,True,-1,1996,682,116


In [51]:
# удаляем дубликаты

# -----------------
# 12 - Acropora_digitifera__XP_015766187.1
# 11 - Acropora_millepora__XP_029179300.2
# 7 - Montipora_foliosa__XP_068703612.1
# 6 - Pocillopora_damicornis__XP_027046452.1
# 9 - Pocillopora_verrucosa__XP_058954563.2
# 2, 4, 5 - Porites_lutea__XP_073247758.1, Porites_lutea__XP_073247755.1, Porites_lutea__XP_073247757.1
# -----------------

indices_to_drop = [12, 11, 7, 6, 9, 2, 4, 5]
df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [52]:
# создаем папку и файлы для выравнивания
os.makedirs(f"../Alignment/psi_blast/{phylum}", exist_ok=True)

# закидываем в конец словаря референсную последовательность (вбить вручную)
dict_align_cds_cassette["Nematostella_vectensis_reference"] = read_single_fasta("../Datasets/Anthozoa/nematostella_vectensis_2/ncbi_dataset/data/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.aln")

# закомментить последнюю строчку после выполнения!!!

In [53]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/psi_blast_{phylum}_cds_cassette.tsv", sep="\t", index=False)